In [262]:
import zipfile
import pandas as pd
import sys

In [263]:
GTFS_PATH = '/home/moritz/dev/uni/multi_modal_data/Helsinki/GTFS.zip'
# GTFS_PATH = '/home/moritz/dev/uni/multi_modal_data/illinois/gtfs_data/chicago_transit_authority_(cta).zip'

# Read data 

In [250]:
STOPS_FILE = "stops.txt"
ROUTES_FILE = "routes.txt"
TRIPS_FILE = "trips.txt"
STOP_TIMES_FILE = "stop_times.txt"
TRANSFERS_FILE = "transfers.txt"

EXPECTED_FILES = [STOPS_FILE, ROUTES_FILE, TRIPS_FILE, STOP_TIMES_FILE, TRANSFERS_FILE]

In [251]:
dfs = ()

with zipfile.ZipFile(GTFS_PATH, 'r') as zip_ref:
	contained = zip_ref.namelist()
	if not all([file in contained for file in EXPECTED_FILES]):
		raise Exception('Not all expected files are in the zip file')

	for file in EXPECTED_FILES:
		with zip_ref.open(file) as f:
			df = pd.read_csv(f)
			dfs += (df,)
	

## Preprocessing
In our algorithms we expect that routes always have the same path, that is the same sequence of stops.
In GTFS this is not given. Routes can go in opposite directions and have different stops.  
In this preprocessing step we make sure that all routes have the same path.  

After we are done the new `routes_id` will have the following format:  
`<old_route_id>_<direction>_<path_id>`


In [252]:
stops_df, routes_df, trips_df, stop_times_df, transfers_df = dfs

In [253]:
# split routes that go into opposite directions
trips_df["old_route_id"] = trips_df["route_id"]
trips_df["route_id"] = trips_df["route_id"] + "_" + trips_df["direction_id"].astype(str)

In [254]:
trips_stop_times_df = pd.merge(trips_df, stop_times_df, on="trip_id")
# create a df that contains the route_id, trip_id and a the ordered list of stop_ids as a string
paths_df = (
    trips_stop_times_df.sort_values(["route_id", "trip_id", "stop_sequence"])
    .groupby(["route_id", "trip_id"])["stop_id"]
    .apply(list)
    .apply(str)
    .reset_index()
)

paths_df = paths_df.rename(columns={"stop_id": "path"})

In [255]:
# now we go through each route and trip and assign a unique path id,
# which is then used in the new route id.
known_route_paths = {}
path_counter_per_route = {}
path_id_by_path = {}

paths_df["new_route_id"] = paths_df["route_id"]

for i,row in paths_df.iterrows():
	path_counter = path_counter_per_route.get(row["route_id"], 0)
	known_paths = known_route_paths.get(row["route_id"], set())

	path_id = None

	path = row["path"]
	if path in known_paths:
		path_id = path_id_by_path[path]
	else:
		path_id = chr(ord('A') + path_counter)
		paths_df.at[i, "new_route_id"] = row["route_id"] + "_" + path_id

		known_paths.add(path)
		known_route_paths[row["route_id"]] = known_paths

		path_counter_per_route[row["route_id"]] = path_counter + 1

		path_id_by_path[path] = path_id

	paths_df.at[i, "new_route_id"] = row["route_id"] + "_" + path_id


In [256]:
trips_df = trips_df.merge(
    paths_df.drop(columns=["path"]), on=["route_id", "trip_id"]
)


In [257]:
trips_df["route_id"] = trips_df["new_route_id"]
trips_df = trips_df.drop(columns=["new_route_id"])

In [258]:
trips_df

,route_id,service_id,trip_headsign,direction_id,shape_id,trip_id,wheelchair_accessible,bikes_allowed,old_route_id
0,1001_1_A,1006H6_20220204_20220320_Ti,Eira,1,1001 6_20211004_2,1001 6_20220204_Ti_2_0514,1,2,1001
1,1001_1_A,1006H6_20220204_20220320_Ti,Eira,1,1001 6_20211004_2,1001 6_20220204_Ti_2_0610,1,2,1001
2,1001_1_B,1006H6_20220204_20220320_Ti,Eira,1,1001 7_20210712_2,1001 7_20220204_Ti_2_0541,1,2,1001
3,1001_1_C,1006H6_20220204_20220320_Ti,Eira,1,1001_20211004_2,1001_20220204_Ti_2_0545,1,2,1001
4,1001_1_C,1006H6_20220204_20220320_Ti,Eira,1,1001_20211004_2,1001_20220204_Ti_2_0557,1,2,1001
...,...,...,...,...,...,...,...,...,...
3190,9788_1_A,1006H6_20220204_20220320_Ti,Rautatientori,1,9788_20210816_2,9788_20220221_Ti_2_1518,1,2,9788
3191,9788_1_A,1006H6_20220204_20220320_Ti,Rautatientori,1,9788_20210816_2,9788_20220221_Ti_2_1617,1,2,9788
3192,9788K_0_A,1006H6_20220204_20220320_Ti,Porvoo,0,9788K_20210816_1,9788K_20220221_Ti_1_0715,1,2,9788K
3193,9788K_0_A,1006H6_20220204_20220320_Ti,Porvoo,0,9788K_20210816_1,9788K_20220221_Ti_1_1242,1,2,9788K


In [259]:
# add first stop id to trips
first_stop_times = (
    stop_times_df.sort_values(["trip_id", "stop_sequence"])
    .groupby("trip_id")
    .first()[["stop_id", "departure_time"]]
    .rename(
        columns={"stop_id": "first_stop_id", "departure_time": "trip_departure_time"}
    )
)

trips_df = trips_df.merge(
    first_stop_times, left_on="trip_id", right_index=True, how="left"
)


In [261]:
stop_times_df

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,1006H6_20220204_Ti_1_1846,18:46:00,18:46:00,1204401,1,Koskelan halli via Rautatieas.,NaN,1.0,0.000,NaN
1,1006H6_20220204_Ti_1_1846,18:48:00,18:48:00,1204403,2,Koskelan halli via Rautatieas.,NaN,NaN,0.524,0.0
2,1006H6_20220204_Ti_1_1846,18:49:00,18:49:00,1204405,3,Koskelan halli via Rautatieas.,NaN,NaN,0.854,0.0
3,1006H6_20220204_Ti_1_1846,18:50:00,18:50:00,1050432,4,Koskelan halli via Rautatieas.,NaN,NaN,1.270,0.0
4,1006H6_20220204_Ti_1_1846,18:52:00,18:52:00,1040440,5,Koskelan halli via Rautatieas.,NaN,NaN,1.538,0.0
...,...,...,...,...,...,...,...,...,...,...
62480,1701_20220204_Ti_2_1000_2,13:41:00,13:42:00,1383192,44,Viikki,NaN,NaN,16.236,NaN
62481,1701_20220204_Ti_2_1000_2,13:44:00,13:44:00,1361116,46,Viikki,NaN,NaN,17.142,0.0
62482,1701_20220204_Ti_2_1000_2,13:45:00,13:45:00,1361114,47,Viikki,NaN,NaN,17.575,0.0
62483,1701_20220204_Ti_2_1000_2,13:45:00,13:45:00,1361112,48,Viikki,NaN,NaN,17.785,0.0


## Convert data from Dataframes into convenient data structures 

In [264]:
# testing
trips_df = pd.read_csv("./tmp/trips.csv")
stop_times_df = pd.read_csv("./tmp/stop_times.csv")

In [266]:
routes_by_stop = {} # l.12
stops_by_route = {} # atm we don't need the stop sequence
stops_by_route_dict = {} # l. 14

stop_times_by_trips_by_route = {}
stop_times_by_trip = {} # atm not needed in this form
times_by_stop_by_trip = {} # l. 24

trip_by_route = {} # trips ordered by first departure time
footpaths_by_route = {} # l. 35

In [267]:
stop_times_by_trip_df = stop_times_df.groupby("trip_id").apply(lambda x: x.sort_values("stop_sequence"))[
    ["arrival_time", "departure_time", "stop_id", "stop_sequence"]
]
for (trip_id, _), data in stop_times_by_trip_df.to_dict('index').items():
    stop_times = stop_times_by_trip.get(trip_id, [])
    stop_times.append(data)
    stop_times_by_trip[trip_id] = stop_times

In [268]:
# stop_id_set = set(stops_df['stop_id']) # we define the stop_id_set later, not through stops_df but rather through actually used stops
route_id_set = set(trips_df['route_id'])
trip_id_set = set(trips_df['trip_id'])

In [292]:
trip_ids_by_route = (
    trips_df.groupby("route_id")["trip_id"]
    .apply(lambda x: x.tolist())
    .to_dict()
)

In [270]:
for route_id, trip_ids in trip_ids_by_route.items():
	stops_ordered = []
	stops = set() # we only need the ordered stops, but use the set to check for duplicates
	for trip_id in trip_ids:
		trip_stop_times = stop_times_by_trip[trip_id]

		stop_times_by_trip_dict = stop_times_by_trips_by_route.get(route_id, {})
		stop_times_by_trip_dict[trip_id] = trip_stop_times
		stop_times_by_trips_by_route[route_id] = stop_times_by_trip_dict

		for stop_time in trip_stop_times:
			stop_in_route = (stop_time['stop_id'], stop_time['stop_sequence'])
			if stop_in_route not in stops:
				stops_ordered.append(stop_in_route)
				stops.add(stop_in_route)

	stops_by_route[route_id] = stops_ordered

In [271]:
# inverse routes_by_stop
routes_by_stop = {}
for route_id, stops in stops_by_route.items():
	for (stop,_) in stops:
		routes = routes_by_stop.get(stop, [])
		routes.append(route_id)
		routes_by_stop[stop] = routes


def get_routes_serving_stop(stop_id):
	return routes_by_stop[stop_id]

In [272]:
stop_id_set = set(routes_by_stop.keys()) # only use stops that are actually used
stops_df = stops_df[stops_df['stop_id'].isin(stop_id_set)] # we use stops_df later, so we need to filter it here

In [293]:
# if we have route and stop, we fastly want to access the stop_sequence
stops_by_route_dict = {
    k: {stop: stop_seq for (stop, stop_seq) in (v)} for k, v in stops_by_route.items()
}

def get_idx_of_stop_in_route(stop_id, route_id):
    return stops_by_route_dict[route_id][stop_id]

In [274]:
def str_time_to_seconds(str_time: str) -> int:
    hours, minutes, seconds = map(int, str_time.split(":"))
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

In [275]:
def seconds_to_str_time(seconds: int) -> str:
    hours = seconds // 3600
    minutes = (seconds - hours * 3600) // 60
    seconds = seconds - hours * 3600 - minutes * 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [276]:
times_by_stop_by_trip = {
    trip_id: {
        stop["stop_id"]: (
            str_time_to_seconds(stop["arrival_time"]),
            str_time_to_seconds(stop["departure_time"]),
        )
        for stop in stops
    }
    for (trip_id, stops) in stop_times_by_trip.items()
}


def get_arrival_time(trip_id: str, stop_id: str) -> int:
    assert trip_id is not None
    assert stop_id is not None

    arrival_time = times_by_stop_by_trip[trip_id][stop_id][0]
    assert type(arrival_time) == int
    return arrival_time



def get_departure_time(trip_id: str, stop_id: str) -> int:
    assert stop_id is not None

    if trip_id is None:
        return sys.maxsize

    departure_time = times_by_stop_by_trip[trip_id][stop_id][1]
    assert type(departure_time) == int
    return departure_time


In [297]:
trip_by_route = {}  # trips ordered by first departure time
trip_by_route = (
    trips_df.sort_values(["route_id", "trip_departure_time"])
    .groupby("route_id")["trip_id"]
    .apply(list)
    .to_dict()
)


## Calculate footpath durations

In [278]:
from pyrosm.data import sources, get_data
import shutil
import os

city = "Helsinki"
path = f"./osm/{city}.pbf"

if not os.path.exists(path):
	fp = get_data(city, update=True)
	os.makedirs("./osm", exist_ok=True)
	shutil.move(fp, path)

In [25]:
import pyrosm

osm = pyrosm.OSM(path)

nodes, edges = osm.get_network(nodes=True, network_type="walking")

In [26]:
import geopandas as gpd
from shapely.geometry import MultiPoint

def trim_osm_to_stops(nodes: gpd.GeoDataFrame, edges: gpd.GeoDataFrame, stops_df: gpd.GeoDataFrame):
	combined_geometry = MultiPoint(stops_df.geometry.tolist())
	convex_hull = combined_geometry.convex_hull
	zone_of_interest = convex_hull.buffer(0.01)

	n_nodes_before = len(nodes)
	n_edges_before = len(edges)

	nodes = nodes.loc[nodes.geometry.within(zone_of_interest), :]
	edges = edges.loc[edges.u.isin(nodes.id) & edges.v.isin(nodes.id), :]
	print(f"""
	{len(nodes)}/{n_nodes_before} ({len(nodes)/n_nodes_before*100:.2f}%) nodes remaining
	{len(edges)}/{n_edges_before} ({len(edges)/n_edges_before*100:.2f}%) edges remaining
	""")

	return nodes, edges

In [27]:
stops_gdf = gpd.GeoDataFrame(
	stops_df, geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat)
)

In [28]:
nodes, edges = trim_osm_to_stops(nodes, edges, stops_gdf)


	255110/973995 (26.19%) nodes remaining
	293374/1095796 (26.77%) edges remaining
	


In [29]:
G_nx = osm.to_graph(nodes, edges, graph_type="networkx")
G_i_graph = osm.to_graph(nodes, edges, graph_type="igraph")

In [30]:
import osmnx as ox
import networkx as nx

In [31]:
stop_ids_df = stops_df[["stop_id", "stop_lat", "stop_lon"]].copy()
nodes, dists = ox.nearest_nodes(
    G_nx, stop_ids_df.stop_lon, stop_ids_df.stop_lat, return_dist=True
)
stop_ids_df["nearest_node"] = nodes
stop_ids_df["nearest_node_dist"] = dists
stop_ids_df.head(2)

,stop_id,stop_lat,stop_lon,nearest_node,nearest_node_dist
0,1240134,60.20583,24.96293,5826933348,0.762623
1,1240133,60.20488,24.96385,772801262,4.249293


In [32]:
print(stop_ids_df.nearest_node_dist.max())

47.29024306378295


In [33]:
stop_ids_df.to_csv("stop_ids_near_nodes.csv", index=False)

In [34]:
stop_ids_gdf = gpd.GeoDataFrame(
    stop_ids_df,
    geometry=gpd.points_from_xy(stop_ids_df.stop_lon, stop_ids_df.stop_lat),
    crs="epsg:4326",
).to_crs("EPSG:32634")
# find stops that are within max_walking_distance beeline distance from each other
avg_walk_speed = 1.4  # m/s
max_walking_duration = 10 * 60  # seconds
max_walking_distance = avg_walk_speed * max_walking_duration
max_walking_distance

nearby_stops_map = {}
for i, row in stop_ids_gdf.iterrows():
    nearby_stops = stop_ids_gdf[
        stop_ids_gdf.geometry.distance(row.geometry) < max_walking_distance
    ].stop_id.tolist()
    # remove self
    nearby_stops = [stop_id for stop_id in nearby_stops if stop_id != row.stop_id]
    nearby_stops_map[row.stop_id] = nearby_stops

In [35]:
node_id_to_g_igraph_node_id_map = {
    node.attributes()["id"]: node.attributes()["node_id"]
    for node in G_i_graph.vs
}

In [36]:
def get_shortest_path_length_igraph(s_node_id, t_node_id):
    paths = G_i_graph.get_shortest_paths(
        node_id_to_g_igraph_node_id_map[s_node_id],
        node_id_to_g_igraph_node_id_map[t_node_id],
        weights="length",
        output="epath",
    )
    path = paths[0]
    return sum(G_i_graph.es[epath]["length"] for epath in path)

In [37]:
nearest_node_dict = stop_ids_gdf.set_index("stop_id")["nearest_node"].to_dict()

In [38]:
from tqdm.notebook import tqdm
from tqdm.contrib.concurrent import process_map

nearby_stops_with_distance_map = {}


def ge(source_stop, nearby_stops):
    return {
        target_stop: get_shortest_path_length_igraph(
            nearest_node_dict[source_stop], nearest_node_dict[target_stop]
        )
        for target_stop in nearby_stops
    }


sources = nearby_stops_map.keys()
res = process_map(
    ge, sources, nearby_stops_map.values(), chunksize=5, max_workers=12
)

for source_stop, nearby_stops_with_distance in zip(sources, res):
	nearby_stops_with_distance_map[source_stop] = nearby_stops_with_distance


# for source_stop, nearby_stops in tqdm(nearby_stops_map.items()):
# 	nearby_stops_with_distance_map[source_stop] = {
# 		target_stop: get_shortest_path_length_igraph(
# 			nearest_node_dict[source_stop], nearest_node_dict[target_stop]
# 		)
# 		for target_stop in nearby_stops
# 	}


  0%|          | 0/1388 [00:00<?, ?it/s]

In [39]:
nearby_stop_with_walking_time_map = {}
for source_stop, nearby_stops_with_distance in nearby_stops_with_distance_map.items():
    	nearby_stop_with_walking_time_map[source_stop] = {
		target_stop: distance / avg_walk_speed
		for target_stop, distance in nearby_stops_with_distance.items()
	}

# Algorithm

In [40]:
import sys

### Helpers

In [41]:
def earliest_trip(route_id: str, stop_id: str, arrival_time: int, change_time: int):
    trip_ids = trip_by_route[route_id]
    for trip_id in trip_ids:
        if get_departure_time(trip_id, stop_id) >= arrival_time + change_time:
            return trip_id

### Assertions

In [ ]:
for stop_id, nearby_stops_with_time in nearby_stop_with_walking_time_map.items():
	for nearby_stop_id, walking_time in nearby_stops_with_time.items():
		assert type(walking_time) == int, f"walking_time is not int: {walking_time}"

### Config

In [214]:
MAX_TRANSFERS = 8
DEFAULT_TRANSFER_TIME = 60 # seconds

start_stop_id = 1040409
end_stop_id = 1361117

start_time = str_time_to_seconds("15:00:00")

### Algorithm

In [215]:
# only needed once

# nearby_stop_with_walking_time_map
# # filter out stops that are not in stop_id_set
# nearby_stop_with_walking_time_map = {
#     source_stop: {
#         target_stop: int(walking_time)
#         for target_stop, walking_time in nearby_stops_with_walking_time.items()
#         if target_stop in stop_id_set and source_stop in stop_id_set
#     }
#     for source_stop, nearby_stops_with_walking_time in nearby_stop_with_walking_time_map.items()
# }


In [216]:
# initialize
tau_i = {
    0: {},
}
tau_best = {}
marked_stops = set()

for stop_id in stop_id_set:
    tau_i[0][stop_id] = sys.maxsize
    tau_best[stop_id] = sys.maxsize

tau_i[0][start_stop_id] = start_time
tau_best[start_stop_id] = start_time

marked_stops.add(start_stop_id)


for k in range(1, MAX_TRANSFERS + 1):
    Q = {}
    tau_i[k] = tau_i[k - 1].copy()

    for stop_id in marked_stops:
        for route_id in get_routes_serving_stop(stop_id):
            if route_id not in Q:
                Q[route_id] = stop_id
                continue

            # if our stop is closer to the start than the existing one, we replace it
            existing_stop_id = Q[route_id]
            idx = get_idx_of_stop_in_route(stop_id, route_id)
            existing_idx = get_idx_of_stop_in_route(existing_stop_id, route_id)
            if idx < existing_idx:
                Q[route_id] = stop_id

    marked_stops = set()

    print(Q)

    for route_id, stop_id in Q.items():
        trip_id = None
        for stop_id, _ in stops_by_route[route_id]:
            if trip_id is not None and get_arrival_time(trip_id, stop_id) < min(
                tau_best[stop_id], tau_best[end_stop_id]
            ):
                arrival_time = get_arrival_time(trip_id, stop_id)
                tau_i[k][stop_id] = arrival_time
                tau_best[stop_id] = arrival_time
                marked_stops.add(stop_id)

            ready_to_depart = tau_i[k - 1][stop_id] + DEFAULT_TRANSFER_TIME
            if ready_to_depart < get_departure_time(trip_id, stop_id):
                trip_id = earliest_trip(route_id, stop_id, ready_to_depart, DEFAULT_TRANSFER_TIME)

    additional_marked_stops = set()
    for stop_id in marked_stops:
        if stop_id == 1040290:
            print("here")
            print(nearby_stop_with_walking_time_map[stop_id])
        for nearby_stop_id, walking_time in nearby_stop_with_walking_time_map[stop_id].items():
            reachable_by_foot = tau_i[k][stop_id] + walking_time
            assert type(reachable_by_foot) == int, f"reachable_by_foot is not int: {reachable_by_foot} type: {str(type(reachable_by_foot))}"

            # if stop_id == 1040290 and nearby_stop_id == 1040411:
            #     print(nearby_stop_id, reachable_by_foot, tau_i[k][nearby_stop_id])

            # we have a couple of modifications here:
            # 1. we only mark the stop if tau is actually updated
            # 2. we use tau_best instead of tau_k (should not make a difference)
            # 3. we also consider tau_best[end_stop_id] just like in the stop before
            # 4. we also update tau_best
            if reachable_by_foot < min(tau_best[nearby_stop_id], tau_best[end_stop_id]):
                tau_i[k][nearby_stop_id] = reachable_by_foot
                tau_best[nearby_stop_id] = reachable_by_foot
                additional_marked_stops.add(nearby_stop_id)
    marked_stops.update(additional_marked_stops)
    
print(tau_best[end_stop_id])

{'1007_1_A': 1040409, '1007_1_B': 1040409, '1009_1_A': 1040409, '1009_1_B': 1040409}
{'1021N_1_A': 1201132, '1024_1_A': 1130183, '1025_1_A': 1130119, '1037_1_A': 1130139, '1041_1_A': 1130139, '1042_1_A': 1130139, '1069_1_A': 1130139, '1070_1_A': 1130139, '2108N_1_A': 1130139, '1021N_0_A': 1040117, '1020_0_A': 1050106, '1030_0_A': 1050106, '1001H_0_A': 1050417, '1001H_0_B': 1050417, '1001_0_B': 1050417, '1003H_0_A': 1050413, '1003_0_A': 1050413, '1006H_0_A': 1204401, '1006_0_B': 1204401, '1022_0_A': 1050105, '1022_1_A': 1201132, '1007_1_A': 1040411, '1007_1_B': 1040411, '1009_1_A': 1040411, '1009_1_B': 1040411, '1007H_0_A': 1203418, '1007_0_B': 1203418, '1009H_0_A': 1203416, '1009_0_A': 1203416, '1001_1_A': 1130439, '1001_1_B': 1130439, '1001_1_C': 1130439, '1002_1_A': 1130439, '1002_1_B': 1130439, '1002_1_C': 1130439, '1004S_1_A': 1020443, '1004_1_A': 1020443, '1004_1_B': 1020443, '1004_1_C': 1020443, '1005_1_A': 1020443, '1010_1_A': 1020443, '1010_1_B': 1020443, '1010_1_C': 1020443, '

In [217]:
for tau in tau_best.values():
	assert type(tau) == int, f"tau is not int: {tau}"

In [218]:
reached = {k: v for k, v in tau_best.items() if v < sys.maxsize}

print(f"reached {len(reached)}/{len(tau_best)} stops ({len(reached)/len(tau_best)*100:.2f}%)")

reached 1355/1361 stops (99.56%)


In [219]:
reached = {k: seconds_to_str_time(v) for k, v in reached.items()}
reached_df = pd.DataFrame.from_dict(reached, orient="index", columns=["time"])

In [220]:
reached_df.index.name = "stop_id"
reached_df.head(2)

,time
stop_id,
1171457,16:36:43
1171458,16:36:00


In [221]:
reached_df.to_csv("reached.csv")